### Reading Data

In [1]:
!pip install gmplot

In [2]:
!pip install gmaps

In [3]:
!pip install unicode

In [4]:
from pandas import Series, DataFrame
import pandas as pd
import numpy
from bokeh.models import LabelSet, ColumnDataSource
from math import pi
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.palettes import Spectral10
from bokeh.transform import factor_cmap
from gmplot import gmplot
import gmaps
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


df = pd.read_csv('/Users/maverick/Documents/MSBA/MSBA SUMMER SEM/HW-python/project/Crime_cleaned.csv')




### Cleaning APD Sector for entire data

In [5]:
df['Incident Number']=df['Incident Number'].astype(str)

In [6]:
indexNames=df[(df['APD Sector'] == 'I') | (df['APD Sector'] == '88') | (df['APD Sector'] == '2') | (df['APD Sector'] == '1')].index

df.drop(indexNames , inplace=True)

df=df.dropna()

df['APD Sector'].value_counts()

list_of_sectors=df['APD Sector'].dropna().unique()



df_2018=df[df['Incident Number'].apply(lambda x:x[0:4])=='2018']

df_2018_APD_values=df_2018['APD Sector'].value_counts()

DAVID      53083
EDWARD     48698
FRANK      44086
BAKER      42920
ADAM       42582
IDA        41950
HENRY      38127
CHARLIE    29277
GEORGE     25288
APD         1164
UT             3
Name: APD Sector, dtype: int64

### Count of Crime in each APD sector in 2018

In [7]:
df_2018_APD_values

DAVID      5078
ADAM       4786
EDWARD     4325
BAKER      4046
FRANK      3892
IDA        3549
HENRY      3376
GEORGE     2634
CHARLIE    2571
APD         155
Name: APD Sector, dtype: int64

### Plot Pie Chart of Top 10 Crime Count in each APD Sector

In [8]:
def plot_pie(x,y):
    
    output_file("pie.html")
    
    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'sector'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Category20c[len(x)]

    p = figure(plot_height=550, plot_width=850,title=y, toolbar_location=None,
               tools="hover", tooltips="@sector: @value", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.45, 
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend='sector', source=data)
    
    data["value"] = data['value'].astype(str)
    data["value"] = data["value"].str.pad(35, side = "left")
    source = ColumnDataSource(data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None

    show(p)

#grouped=df_2018.groupby('APD Sector')

#x=(grouped['Highest Offense Description'].get_group('ADAM').value_counts()[:10])

#plot_pie(x,'ADAM')

In [10]:
def plot_pie_charts(df):

    grouped=df_2018.groupby('APD Sector')
    for name,group in grouped:
        print(name)
        print(group['Highest Offense Description'].value_counts()[:10].plot(kind='bar'))
        %matplotlib inline
        
       
    df['APD Sector'].values

    for each_sector in df_2018['APD Sector'].unique():
           
        x=(grouped['Highest Offense Description'].get_group(each_sector).value_counts()[:10])
        
        plot_pie(x,each_sector)
               

plot_pie_charts(df_2018)


ADAM
AxesSubplot(0.125,0.125;0.775x0.755)
APD
AxesSubplot(0.125,0.125;0.775x0.755)
BAKER
AxesSubplot(0.125,0.125;0.775x0.755)
CHARLIE
AxesSubplot(0.125,0.125;0.775x0.755)
DAVID
AxesSubplot(0.125,0.125;0.775x0.755)
EDWARD
AxesSubplot(0.125,0.125;0.775x0.755)
FRANK
AxesSubplot(0.125,0.125;0.775x0.755)
GEORGE
AxesSubplot(0.125,0.125;0.775x0.755)
HENRY
AxesSubplot(0.125,0.125;0.775x0.755)
IDA
AxesSubplot(0.125,0.125;0.775x0.755)


### Plot Bar Graphs for Crime Count per Population for each APD Sector in 2018

In [12]:
def plot_bar(x,y,z):
    
    if z=='violent':
        output_file("Violent_bars.html")
    elif z=='non-violent':
        output_file("Non-Violent_bars.html")
    else:
        output_file("total_bars.html")
    source = ColumnDataSource(data=dict(fruits=y, counts=x))
    
    p = figure(x_range=y, plot_height=600,plot_width=900, toolbar_location=None, title="")
    p.vbar(x='fruits', top='counts', width=0.8, source=source, legend="fruits",
           line_color='white', fill_color=factor_cmap('fruits', palette=Spectral10, factors=y))

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    #p.y_range.end = 9
    p.legend.orientation = "horizontal"
    p.legend.location = "top_center"
    
    show(p)

In [13]:
def plot_bar_graphs(df):

    categories= [['Murder','Aggravated Assault','Rape'],['Burglary','Robbery','Theft','Auto Theft']]
   
    #Population of each APD Sector
    ind=[123999,11835,61607,137527,100894,60120,66059,139085,132014]
    
    #List of APD Sectors
    lst=['BAKER','GEORGE','IDA','ADAM','EDWARD','CHARLIE','HENRY','DAVID','FRANK']
    
    population = pd.Series(ind, index = lst) 
    
    #Violent Categories include Murder, Aggravated Assault and Rape
    violent= df[df['Category Description'].isin(categories[0])]['APD Sector']
    violent= df[df['Category Description'].isin(categories[0])]['APD Sector'].value_counts(ascending=True)
    
    #Non-Violent Categories include Burglary, Robbery, Theft and Auto Theft
    non_violent = df[df['Category Description'].isin(categories[1])]
    non_violent= df[df['Category Description'].isin(categories[1])]['APD Sector'].value_counts(ascending=True)[1:]  
    
    total_count= df_2018['APD Sector'].value_counts(ascending=True)[1:]  
    
    #Plot Pie Charts of Violent Crime depending on APD Sector
    plot_pie(violent,'Violent crimes')
    
    #Plot Pie Charts of Non-Violent Crime depending on APD Sector
    plot_pie(non_violent,'Non Violent Crimes')
    
    plot_bar((violent/population).sort_values(ascending=True)*100,(violent/population).sort_values(ascending=True).index.values,'violent')
    plot_bar((non_violent/population).sort_values(ascending=True)*100,(non_violent/population).sort_values(ascending=True).index.values,'non-violent')
    plot_bar(total_count.sort_values(ascending=True),total_count.sort_values(ascending=True).index.values,'total')

plot_bar_graphs(df_2018)


### Heat-Maps of Crime in Austin 2018

In [15]:
def create_map(data):
    
    data.dropna()
    min_lat, max_lat, min_lon, max_lon = \
    float(min(data['Latitude'])), float(max(data['Latitude'])), \
    float(min(data['Longitude'])), float(max(data['Longitude']))

    ## Create empty map with zoom level 16

    mymap = gmplot.GoogleMapPlotter(
    min_lat + (max_lat - min_lat) / 2, 
    min_lon + (max_lon - min_lon) / 2, 
    16)

   # mymap.configureapikey="EnterAPI"
    
    grouped=df.dropna().groupby('Category Description')
   
    violent=['Murder','Aggravated Assault']
    non_violent=['Burglary','Robbery','Theft','Auto Theft']
    
    for each_category in data['Category Description'].dropna().unique():        
         
            sector_sort=grouped.get_group(each_category)['APD Sector'].dropna().value_counts().index.values
            
            mymap.heatmap(grouped.get_group(each_category)['Latitude'].astype(float),grouped.get_group(each_category)['Longitude'].astype(float),radius=15, opacity=0.5)
           # mymap.scatter(grouped.get_group(each_category)[:100]['Latitude'].astype(float),grouped.get_group(each_category)['Longitude'][:100].astype(float), c='r', marker=False)
            ####mymap.heatmap(float(grouped.get_group(each_category).iloc[0]['Latitude']), float(grouped.get_group(each_category).iloc[0]['Longitude']))

            mymap.draw('Heat_Map'+ each_category + '.html')
            print('Heat_Map'+ each_category + '.html')
    
create_map(df_2018)


Heat_MapBurglary.html
Heat_MapRobbery.html
Heat_MapTheft.html
Heat_MapAggravated Assault.html
Heat_MapAuto Theft.html
Heat_MapMurder.html
